# Torah Analyzer (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2024

***

In [ ]:
# @title NVIDA GPU Check
!nvidia-smi

In [ ]:
# @title Install requirements
!git clone https://github.com/asigalov61/tegridy-tools
!pip install transformers -U
!pip install accelerate -U
!pip install gradio -U

In [ ]:
# @title Import needed modules

print('=' * 70)
print('Loading modules...')
print('=' * 70)

%cd /content/tegridy-tools/tegridy-tools/

import TORAH

%cd /content/

import torch

from transformers import AutoTokenizer, AutoModelForCausalLM

import gradio as gr

print('=' * 70)
print('Done!')
print('=' * 70)

In [ ]:
# @title Load/Reload Mistral 7B Instruct model

#@markdown https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map='cuda')

In [ ]:
# @title Start/Restart Torah Analyzer Gradio interface

questions = [
            'None',
            'Please summarize the following text. Thank you.',
            'Please write fifty questions about the provided text. Thank you.',
            'Please write ten summary questions in order of importance. Thank you.',
            'Please write twenty questions about the text in order of significance. Thank you.',
            'Please write a short comprehensive essay about the most significant and important points of the text. Thank you.',
            'Please write a short comprehensive literary summary essay about the most significant and important points of the text. Thank you.',
            'Please write a short fictional essay based on the text. Thank you.',
            'Please write a very long fictional essay based on the text. Thank you.',
            'Please write a rephrased version of the provided text. Thank you.',
            'Please write ten literary fictional titles for the provided text. Thank you.',
            'Please write all possible questions about the provided text. Thank you.',
            'Please write two hundred questions about the provided text. Thank you.',
            'Please write ten most subtle, elusive, and tenious details from the provided text. Thank you.',
            'Please write detailed statistical and numerical analysis of the provided text. Thank you.',
            'Please write a single sentence summary of the provided text. Thank you.',
            'Please write ten keywords of the text in order of importance and significance. Thank you.',
            'What is the most important point of the text? Thank you.',
            'What are the key numbers in the text? Thank you.',
            'What are the key points of the text? Thank you.',
            'What are the keywords of the text? Thank you.',
            'What is the most significant point of the text?. Thank you.',
            'What is the most intricate and delicate point in the provided text? Thank you.',

            ]

with gr.Blocks() as demo:

    def fetch_torah_portion(torah_portion):
      if torah_portion != 'None':
        tp = TORAH.torah_weekly_portion_getter(torah_portion_name=torah_portion)[2]
        return tp

      else:
        return None

    torah_portions = ['None'] + [t[0] for t in TORAH.TORAH_WEEKLY_PORTIONS]

    gr.Markdown("<h1 style='text-align: center; margin-bottom: 1rem'>Torah Analyzer</h1>")
    prompts = gr.Dropdown(torah_portions, label="Select desired Torah portion to talk about", value='None')
    imsg = gr.Textbox(label='Selected Torah portion to analyze')
    chatbot = gr.Chatbot(show_copy_button=True)
    cmsg = gr.Dropdown(questions, label="Select a sample question or request to ask about")
    msg = gr.Textbox(label='Enter your own question or request to ask about the text')
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history, imessage):

        torch.cuda.empty_cache()

        gen_len = 5000
        num_batches = 1
        temperature = 0.9

        instruct_prompt = message

        #========================================================================

        if imessage and message != 'None':

          prompt = '<s>[INST]' + instruct_prompt + '\n' + imessage + '\n' + instruct_prompt + '[/INST]' + '\n'

          input_ids = torch.LongTensor([tokenizer.encode(prompt)] * num_batches).cuda()

          print('=' * 70)
          print('Req len', len(input_ids[0]))
          print('=' * 70)

          output = model.generate(input_ids,
                                  max_length=len(input_ids[0]) + gen_len,
                                  temperature=temperature,
                                  do_sample=True,
                                  pad_token_id=tokenizer.pad_token_id)

          generated_ids = output[:, len(input_ids[0]):]

          for g in generated_ids:
            generated_text = tokenizer.decode(g, skip_special_tokens=True)

          bot_message = generated_text

          chat_history.append((message, bot_message))

          return "", chat_history

        else:
          bot_message = 'No Torah portion was selected to analyze!\nPlease select Torah portion first!'

          chat_history.append((message, bot_message))

          return "", chat_history

    prompts.input(fetch_torah_portion, [prompts], [imsg])
    cmsg.input(respond, [cmsg, chatbot, imsg], [msg, chatbot])

    msg.submit(respond, [msg, chatbot, imsg], [msg, chatbot])


if __name__ == "__main__":
    demo.launch()

# Congrats! You did it! :)